## 1. Setup and Imports


In [1]:
# Import required libraries
import time
import logging
import pandas as pd
import re
from datetime import datetime
from typing import List, Dict, Optional

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup


## 2. Configuration and Logging Setup


In [2]:
# Configuration
BASE_URL = "https://www.myschool.edu.au"
SEARCH_URL = "https://myschool.edu.au/school-search?FormPosted=True&SchoolSearchQuery=&SchoolSector=&SchoolType=&State=Vic&pageNumber={}"
YEARS_TO_SCRAPE = [2023, 2024]
TEST_DOMAINS = ['Reading', 'Writing', 'Spelling', 'Grammar', 'Numeracy']
YEAR_LEVELS = ['Year 3', 'Year 5', 'Year 7', 'Year 9']

# Timing
PAGE_LOAD_TIMEOUT = 30
ELEMENT_WAIT_TIMEOUT = 15
BETWEEN_REQUESTS_DELAY = 2

# Output
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_CSV = "Final_Result.csv"

# Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    handlers=[
        logging.FileHandler(f'naplan_scraper_{timestamp}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


## 3. Utility Functions


In [3]:
def setup_driver():
    """Initialize Chrome WebDriver"""
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
    return driver


def handle_accept_page(driver):
    """Handle MySchool terms of use page"""
    time.sleep(2)
    
    try:
        checkbox = driver.find_element(By.CSS_SELECTOR, "#checkBoxTou")
    except NoSuchElementException:
        return False
    
    # Click checkbox
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", checkbox)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", checkbox)
    
    if not checkbox.is_selected():
        checkbox.click()
    
    time.sleep(1)
    
    # Click accept button
    button_selectors = [
        "button.accept",
        "button.button-link.accept",
        "button[type='submit'].accept",
        "//button[contains(@class, 'accept')]"
    ]
    
    for selector in button_selectors:
        try:
            if selector.startswith("//"):
                button = driver.find_element(By.XPATH, selector)
            else:
                button = driver.find_element(By.CSS_SELECTOR, selector)
            
            if button.is_displayed():
                driver.execute_script("arguments[0].removeAttribute('disabled');", button)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
                logger.info("Terms page accepted")
                return True
        except:
            continue
    
    return False


def extract_school_id_from_url(url):
    """Extract school ID from URL"""
    match = re.search(r'/school/(\d+)', url)
    return match.group(1) if match else None


def clean_numeric_value(value):
    """Clean and convert numeric values"""
    if not value or value.strip() == '':
        return None
    try:
        cleaned = re.sub(r'[^0-9.-]', '', str(value))
        return float(cleaned) if cleaned else None
    except:
        return None


## 3.1. NAPLAN Data Scraping Function

This function scrapes NAPLAN test results from individual school pages.


In [ ]:
def scrape_naplan_data(driver, school_id, year):
    """
    Scrape NAPLAN test results for a specific school and year.
    
    Args:
        driver: Selenium WebDriver instance
        school_id: School identifier
        year: Year to scrape (e.g., 2024)
    
    Returns:
        list: List of dictionaries containing NAPLAN data for each year level
    """
    results = []
    
    try:
        logger.info(f"Scraping NAPLAN data: School {school_id}, Year {year}")
        
        # Wait for JavaScript to render the data
        time.sleep(5)
        
        # Try to find the NAPLAN table using Selenium (which sees rendered content)
        try:
            # Wait for table to be present
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
        except TimeoutException:
            logger.warning(f"Timeout waiting for table")
        
        # Extract data from the rendered page using Selenium elements
        # The table has year levels as rows and subjects as columns
        
        # Find all table rows
        try:
            table_rows = driver.find_elements(By.CSS_SELECTOR, "table tr, tbody tr")
            logger.debug(f"Found {len(table_rows)} table rows")
            
            # Try to find participation percentages in the text
            page_text = driver.find_element(By.TAG_NAME, "body").text
            
            # Extract participation rates
            school_participation = None
            australian_participation = None
            
            # Look for participation text patterns
            if "NAPLAN participation for this school" in page_text:
                school_match = re.search(r'NAPLAN participation for this school is (\d+)%', page_text)
                if school_match:
                    school_participation = int(school_match.group(1))
                    
            if "NAPLAN participation for all Australian students" in page_text:
                australian_match = re.search(r'NAPLAN participation for all Australian students is (\d+)%', page_text)
                if australian_match:
                    australian_participation = int(australian_match.group(1))
            
            # Process each year level
            for year_level in YEAR_LEVELS:
                try:
                    # Look for the year level in the table
                    # Find cells that contain the year level text
                    year_cells = driver.find_elements(By.XPATH, f"//td[contains(text(), '{year_level}')]|//th[contains(text(), '{year_level}')]")
                    
                    if not year_cells:
                        logger.debug(f"{year_level}: Not found on page")
                        continue
                    
                    # Get the row containing this year level
                    year_cell = year_cells[0]
                    row = year_cell.find_element(By.XPATH, "./..")  # Parent tr element
                    
                    # Get all cells in this row
                    cells = row.find_elements(By.TAG_NAME, "td")
                    cell_texts = [cell.text.strip() for cell in cells]
                    
                    logger.debug(f"{year_level}: Found row with {len(cells)} cells: {cell_texts}")
                    
                    # Create record for this year level
                    record = {
                        'school_id': school_id,
                        'year': year,
                        'year_level': year_level,
                        'school_participation_pct': school_participation,
                        'australian_participation_pct': australian_participation
                    }
                    
                    # The table typically has: Year Level | Reading | Writing | Spelling | Grammar | Numeracy
                    # Extract scores by position (skip first cell which is the year level label)
                    score_cells = cell_texts[1:]  # Skip the year level label
                    
                    # Map scores to domains in order
                    for idx, domain in enumerate(TEST_DOMAINS):
                        if idx < len(score_cells):
                            score_text = score_cells[idx]
                            # Try to extract numeric value
                            try:
                                # Handle cases like "455", "NP" (Not Published), "−" (no data)
                                if score_text and score_text not in ['NP', '−', '', '-']:
                                    score = float(score_text)
                                    record[domain] = score
                                else:
                                    record[domain] = None
                            except (ValueError, AttributeError):
                                record[domain] = None
                                logger.debug(f"{year_level} {domain}: Could not parse '{score_text}'")
                        else:
                            record[domain] = None
                    
                    # Only add record if at least one score was found
                    if any(record.get(domain) is not None for domain in TEST_DOMAINS):
                        results.append(record)
                        logger.info(f"  {year_level}: Extracted scores")
                    else:
                        logger.debug(f"{year_level}: No valid scores found")
                        
                except Exception as e:
                    logger.debug(f"Error extracting {year_level}: {e}")
                    continue
            
        except Exception as e:
            logger.warning(f"Error finding table elements: {e}")
        
        if results:
            logger.info(f"Successfully scraped {len(results)} records for school {school_id}, year {year}")
        else:
            logger.warning(f"No NAPLAN data found for school {school_id}, year {year}")
            
    except Exception as e:
        logger.error(f"Error scraping NAPLAN data for school {school_id}, year {year}: {e}")
    
    return results




In [5]:
def scrape_school_list(driver, max_pages=None):
    """
    Scrape the list of Victorian schools from the search page.
    Schools are loaded dynamically, so we need to wait and find "View School Profile" links.
    
    Args:
        driver: Selenium WebDriver instance
        max_pages: Maximum number of pages to scrape (None for all)
    
    Returns:
        list: List of dictionaries containing school information
    """
    schools = []
    page_number = 1
    
    logger.info("Starting school list scraping...")
    logger.info("Note: Schools load dynamically via JavaScript")
    logger.info("Using button clicks for pagination instead of URL navigation")
    
    # Load the first page only
    url = SEARCH_URL.format(1)
    logger.info(f"Loading initial search page: {url}")
    
    try:
        driver.get(url)
        time.sleep(3)
        handle_accept_page(driver)
        logger.info("Waiting for initial schools to load...")
        time.sleep(5)
    except Exception as e:
        logger.error(f"Failed to load initial page: {e}")
        return schools
    
    # Now loop through pages using the Next button
    while True:
        if max_pages and page_number > max_pages:
            logger.info(f"Reached maximum pages limit: {max_pages}")
            break
        
        logger.info(f"Processing page {page_number}...")
        
        try:
            # Look for school containers with class "myschool-search-list-view"
            logger.info("Looking for school containers with class 'myschool-search-list-view'...")
            
            try:
                # Find all school container divs
                school_containers = driver.find_elements(By.CSS_SELECTOR, "div.myschool-search-list-view")
                
                if school_containers:
                    logger.info(f"Found {len(school_containers)} school containers!")
                else:
                    logger.warning("No school containers found with class 'myschool-search-list-view'")
                    
            except Exception as e:
                logger.error(f"Error finding school containers: {e}")
                school_containers = []
            
            if not school_containers:
                logger.error(f"No schools found on page {page_number}")
                logger.error(f"Current URL: {driver.current_url}")
                logger.error(f"Page title: {driver.title}")
                break
            
            # Process each school container
            page_schools = 0
            logger.info(f"Processing {len(school_containers)} school containers...")
            
            for container in school_containers:
                try:
                    # Find the link inside this container (format: /school/{ID})
                    link = container.find_element(By.CSS_SELECTOR, "a[href*='/school/']")
                    school_url = link.get_attribute('href')
                    
                    if not school_url:
                        continue
                    
                    # Extract school ID from URL
                    school_id = extract_school_id_from_url(school_url)
                    if not school_id:
                        logger.debug(f"Could not extract school ID from: {school_url}")
                        continue
                    
                    # Check if we already have this school
                    if any(s['school_id'] == school_id for s in schools):
                        logger.debug(f"School {school_id} already in list, skipping")
                        continue
                    
                    # Get school name from the container text
                    # The container has the full school info, we need to extract just the name
                    container_text = container.text.strip()
                    
                    # School name is typically the first line or before newline
                    if container_text:
                        # Split by newlines and take the first non-empty line
                        lines = [line.strip() for line in container_text.split('\n') if line.strip()]
                        school_name = lines[0] if lines else f"School_{school_id}"
                    else:
                        school_name = f"School_{school_id}"
                    
                    # Create school info dictionary
                    school_info = {
                        'school_id': school_id,
                        'school_name': school_name,
                        'school_url': school_url
                    }
                    
                    schools.append(school_info)
                    page_schools += 1
                    logger.info(f"Added: {school_name} (ID: {school_id})")
                    
                except NoSuchElementException:
                    logger.debug(f"No link found in container, skipping")
                    continue
                except Exception as e:
                    logger.warning(f"Error processing school container: {e}")
                    continue
            
            logger.info(f"Found {page_schools} NEW schools on page {page_number}. Total: {len(schools)}")
            
            # Check if we should continue to next page
            if max_pages and page_number >= max_pages:
                logger.info(f"Reached page limit ({max_pages})")
                break
            
            # Find and click the Next button to go to next page
            try:
                logger.info("Looking for 'Next' button to navigate to next page...")
                
                # Try multiple selectors for the Next button
                next_selectors = [
                    "a[aria-label='Next page']",
                    "button[aria-label='Next page']",
                    "a.next",
                    "button.next",
                    "//a[contains(@class, 'next')]",
                    "//button[contains(@class, 'next')]",
                    "//a[contains(text(), 'Next')]",
                    "//button[contains(text(), 'Next')]",
                ]
                
                next_button = None
                for selector in next_selectors:
                    try:
                        if selector.startswith("//"):
                            btn = driver.find_element(By.XPATH, selector)
                        else:
                            btn = driver.find_element(By.CSS_SELECTOR, selector)
                        
                        if btn and btn.is_displayed() and btn.is_enabled():
                            next_button = btn
                            logger.info(f"Found Next button using selector: {selector}")
                            break
                    except:
                        continue
                
                if not next_button:
                    logger.info("No more pages - Next button not found or disabled")
                    break
                
                # Scroll button into view and click it
                logger.info("Clicking Next button...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                
                # Use JavaScript click for reliability
                driver.execute_script("arguments[0].click();", next_button)
                logger.info("Clicked Next button")
                
                # Wait for new page to load
                page_number += 1
                logger.info(f"Waiting for page {page_number} to load...")
                time.sleep(5)  # Wait for JavaScript to load new schools
                
                # Verify content actually changed by checking first school
                try:
                    first_container = driver.find_element(By.CSS_SELECTOR, "div.myschool-search-list-view")
                    first_link = first_container.find_element(By.CSS_SELECTOR, "a[href*='/school/']")
                    new_first_url = first_link.get_attribute('href')
                    new_first_id = extract_school_id_from_url(new_first_url)
                    
                    # Check if first school is one we already have
                    if any(s['school_id'] == new_first_id for s in schools):
                        logger.warning(f"After clicking Next, first school ({new_first_id}) is already in our list - might be same page")
                    else:
                        logger.info(f"New page loaded successfully, first school: {new_first_id}")
                except:
                    logger.warning("Could not verify new page loaded, continuing anyway...")
                    
            except Exception as e:
                logger.error(f"Error navigating to next page: {e}")
                break
            
        except Exception as e:
            logger.error(f"Error scraping page {page_number}: {e}")
            break
    
    logger.info(f"Completed school list scraping. Total schools found: {len(schools)}")
    return schools




## 4.5. School List Scraper (Names and IDs Only)



In [6]:
def scrape_school_list_only(max_schools=None, max_pages=None):
    """
    Simple function to just scrape school names and IDs from search results.
    Does NOT scrape NAPLAN data - just creates a list of schools.
    
    Args:
        max_schools: Maximum number of schools to collect (None for all)
        max_pages: Maximum number of search pages to scrape (None for all)
    
    Returns:
        pandas.DataFrame: DataFrame with school_id, school_name, school_url
    """
    driver = None
    
    try:
        # Setup driver
        driver = setup_driver()
        logger.info("SIMPLE SCHOOL LIST COLLECTION")
        logger.info("This will only collect school names and IDs")
        logger.info("No NAPLAN data will be scraped in this step")
        
        # Scrape school list
        schools = scrape_school_list(driver, max_pages=max_pages)
        
        if not schools:
            logger.error("No schools found!")
            return pd.DataFrame()
        
        # Apply max_schools limit if specified
        if max_schools:
            schools = schools[:max_schools]
            logger.info(f"Limited to first {max_schools} schools")
        
        logger.info(f"COLLECTION COMPLETE!")
        logger.info(f"Total schools collected: {len(schools)}")
        
    except Exception as e:
        logger.error(f"Error in school collection: {e}")
        return pd.DataFrame()
    
    finally:
        if driver:
            driver.quit()
            logger.info("WebDriver closed")
    
    # Convert to DataFrame
    if schools:
        df = pd.DataFrame(schools)
        return df
    else:
        return pd.DataFrame()




## 4.6. Collect School Names and IDs Only


In [ ]:


# Used 3 pages for school name scraping
## for all school name to be scrapped need to use max_pages = 140

schools_df = scrape_school_list_only(max_schools=None, max_pages=3)

if not schools_df.empty:
    print()
    print("="*60)
    print("="*60)
    print(f"Total schools: {len(schools_df)}")
    print(f"Columns: {schools_df.columns.tolist()}")
    print()
    print("Schools DataFrame:")
    display(schools_df)
    
    # Save to CSV
    schools_df.to_csv("schools_list_test.csv", index=False)
else:
    print("No schools collected")


2025-10-30 14:58:32,854 - ====== WebDriver manager ======


TESTING: Collecting school names and IDs
This will collect schools from first 3 pages
   - Max pages: 3
   - No school limit (will collect all schools from these pages)



2025-10-30 14:58:33,911 - Get LATEST chromedriver version for google-chrome
2025-10-30 14:58:34,427 - Get LATEST chromedriver version for google-chrome
2025-10-30 14:58:34,875 - Driver [C:\Users\CY\.wdm\drivers\chromedriver\win64\141.0.7390.122\chromedriver-win32/chromedriver.exe] found in cache
2025-10-30 14:58:35,997 - SIMPLE SCHOOL LIST COLLECTION
2025-10-30 14:58:35,998 - This will only collect school names and IDs
2025-10-30 14:58:35,999 - No NAPLAN data will be scraped in this step
2025-10-30 14:58:36,000 - Starting school list scraping...
2025-10-30 14:58:36,002 - Note: Schools load dynamically via JavaScript
2025-10-30 14:58:36,003 - Using button clicks for pagination instead of URL navigation
2025-10-30 14:58:36,004 - Loading initial search page: https://myschool.edu.au/school-search?FormPosted=True&SchoolSearchQuery=&SchoolSector=&SchoolType=&State=Vic&pageNumber=1
2025-10-30 14:58:49,005 - Terms page accepted
2025-10-30 14:58:49,006 - Waiting for initial schools to load...
2


Total schools: 52
Columns: ['school_id', 'school_name', 'school_url']

Schools DataFrame:


,school_id,school_name,school_url
0,44370,Abbotsford Primary School,https://myschool.edu.au/school/44370
1,44727,Aberfeldie Primary School,https://myschool.edu.au/school/44727
2,45704,Academy of Mary Immaculate,https://myschool.edu.au/school/45704
3,46213,Adass Israel School,https://myschool.edu.au/school/46213
4,52378,Advance College of Education Incorporated,https://myschool.edu.au/school/52378
5,44886,Ainslie Parklands Primary School,https://myschool.edu.au/school/44886
6,52936,Aintree Primary School,https://myschool.edu.au/school/52936
7,52492,Aireys Inlet Primary School,https://myschool.edu.au/school/52492
8,44716,Airly Primary School,https://myschool.edu.au/school/44716
9,46353,Aitken College,https://myschool.edu.au/school/46353


## 5. NAPLAN Data Collection from School List

Now that we have the school list, use it to scrape NAPLAN data for each school.


In [8]:
def scrape_naplan_for_schools(schools_df, years=None):
    """
    Scrape NAPLAN data for each school in the provided DataFrame.
    
    Args:
        schools_df: DataFrame with columns ['school_id', 'school_name', 'school_url']
        years: List of years to scrape (default: [2022, 2023, 2024])
    
    Returns:
        pandas.DataFrame: Complete NAPLAN data with one row per (School × Year × Year Level)
    """
    if years is None:
        years = YEARS_TO_SCRAPE
    
    driver = None
    all_naplan_data = []
    
    try:
        # Setup driver
        driver = setup_driver()
        
        logger.info("NAPLAN DATA COLLECTION FROM SCHOOL LIST")
        logger.info(f"Schools to process: {len(schools_df)}")
        logger.info(f"Years to scrape: {years}")
        logger.info(f"Expected pages per school: {len(years)}")
        
        total_schools = len(schools_df)
        
        # Process each school
        for idx, row in schools_df.iterrows():
            school_id = row['school_id']
            school_name = row['school_name']
            
            logger.info(f"\n{'='*60}")
            logger.info(f"[{idx+1}/{total_schools}] Processing: {school_name}")
            logger.info(f"School ID: {school_id}")
            logger.info(f"{'='*60}")
            
            # Scrape NAPLAN data for each year
            for year in years:
                try:
                    logger.info(f"  Scraping year {year}...")
                    
                    # Construct NAPLAN URL
                    naplan_url = f"{BASE_URL}/school/{school_id}/naplan/results/{year}"
                    logger.info(f"     URL: {naplan_url}")
                    
                    # Visit the page
                    driver.get(naplan_url)
                    time.sleep(2)
                    
                    # Handle terms page if it appears again
                    handle_accept_page(driver)
                    
                    # Scrape the NAPLAN data
                    results = scrape_naplan_data(driver, school_id, year)
                    
                    if results:
                        # Add school name to each record
                        for record in results:
                            record['school_name'] = school_name
                        
                        all_naplan_data.extend(results)
                        logger.info(f"   Collected {len(results)} records")
                    else:
                        logger.warning(f"   No data found for {year}")
                    
                    # Polite delay between requests
                    time.sleep(BETWEEN_REQUESTS_DELAY)
                    
                except Exception as e:
                    logger.error(f"   Error scraping {school_name} for {year}: {e}")
                    continue
            
            # Progress update every 5 schools
            if (idx + 1) % 5 == 0:
                logger.info(f"\nProgress Update:")
                logger.info(f"   Schools processed: {idx + 1}/{total_schools}")
                logger.info(f"   Records collected: {len(all_naplan_data)}")
                logger.info(f"   Estimated remaining time: ~{(total_schools - idx - 1) * len(years) * 5} seconds")
        
        logger.info("\n" + "="*60)
        logger.info("NAPLAN DATA COLLECTION COMPLETE!")
        logger.info(f"Total schools processed: {total_schools}")
        logger.info(f"Total records collected: {len(all_naplan_data)}")
        logger.info(f"Years covered: {years}")
        
    except Exception as e:
        logger.error(f"Fatal error in NAPLAN data collection: {e}")
        raise
    
    finally:
        if driver:
            driver.quit()
            logger.info("WebDriver closed")
    
    # Convert to DataFrame
    if all_naplan_data:
        df = pd.DataFrame(all_naplan_data)
        
        # Reorder columns for better readability
        column_order = [
            'school_id',
            'school_name',
            'year',
            'year_level',
            'Reading',
            'Writing',
            'Spelling',
            'Grammar',
            'Numeracy',
            'school_participation_pct',
            'australian_participation_pct'
        ]
        
        # Only include columns that exist
        available_cols = [col for col in column_order if col in df.columns]
        df = df[available_cols]
        
        return df
    else:
        logger.warning("No NAPLAN data was collected!")
        return pd.DataFrame()




## 6. Complete Pipeline: Schools + NAPLAN Data



In [ ]:
if 'schools_df' in globals() and not schools_df.empty:
    print(f"  Schools available: {len(schools_df)}")
    print()
    
    schools_to_process = schools_df  # All schools from the test
    
    print(f"Will process {len(schools_to_process)} schools for NAPLAN data")
    print()
else:
    print("   Please run Cell 14 first to collect schools.")
    print()
    schools_to_process = None


  Schools available: 52

Will process 52 schools for NAPLAN data



In [ ]:
if schools_to_process is not None and not schools_to_process.empty:
    print("="*60)
    print("Starting NAPLAN data collection...")
    print("="*60)
    print(f"Schools: {len(schools_to_process)}")
    print(f"Years: {YEARS_TO_SCRAPE}")
    print(f"This may take approximately {len(schools_to_process) * len(YEARS_TO_SCRAPE) * 5} seconds")
    print()
    
    # Run the NAPLAN scraper
    naplan_df = scrape_naplan_for_schools(schools_to_process, years=YEARS_TO_SCRAPE)
    
    # Display results
    if not naplan_df.empty:
        print("\n" + "="*60)
        print("="*60)
        print(f"\nTotal records: {len(naplan_df)}")
        print(f"Schools: {naplan_df['school_id'].nunique()}")
        print(f"Years: {sorted(naplan_df['year'].unique())}")
        print(f"Year levels: {sorted(naplan_df['year_level'].unique())}")
        print(f"\nColumns: {naplan_df.columns.tolist()}")
        
        print("\nFirst 20 rows:")
        display(naplan_df.head(20))
        
        print("\nSummary statistics:")
        print(naplan_df[['Reading', 'Writing', 'Spelling', 'Grammar', 'Numeracy']].describe())
    else:
        print("No NAPLAN data was collected")
else:
    print("Skipping NAPLAN collection - no schools to process")


2025-10-30 14:59:14,786 - ====== WebDriver manager ======


Starting NAPLAN data collection...
Schools: 52
Years: [2023, 2024]
This may take approximately 520 seconds



2025-10-30 14:59:15,767 - Get LATEST chromedriver version for google-chrome
2025-10-30 14:59:16,209 - Get LATEST chromedriver version for google-chrome
2025-10-30 14:59:16,653 - Driver [C:\Users\CY\.wdm\drivers\chromedriver\win64\141.0.7390.122\chromedriver-win32/chromedriver.exe] found in cache
2025-10-30 14:59:17,661 - NAPLAN DATA COLLECTION FROM SCHOOL LIST
2025-10-30 14:59:17,662 - Schools to process: 52
2025-10-30 14:59:17,664 - Years to scrape: [2023, 2024]
2025-10-30 14:59:17,664 - Expected pages per school: 2
2025-10-30 14:59:17,665 - 
2025-10-30 14:59:17,666 - [1/52] Processing: Abbotsford Primary School
2025-10-30 14:59:17,667 - School ID: 44370
2025-10-30 14:59:17,668 - ============================================================
2025-10-30 14:59:17,669 -   Scraping year 2023...
2025-10-30 14:59:17,670 -      URL: https://www.myschool.edu.au/school/44370/naplan/results/2023
2025-10-30 14:59:33,799 -    Error scraping Abbotsford Primary School for 2023: Message: stale element



Total records: 224
Schools: 49
Years: [np.int64(2023), np.int64(2024)]
Year levels: ['Year 3', 'Year 5', 'Year 7', 'Year 9']

Columns: ['school_id', 'school_name', 'year', 'year_level', 'Reading', 'Writing', 'Spelling', 'Grammar', 'Numeracy', 'school_participation_pct', 'australian_participation_pct']

First 20 rows:


,school_id,school_name,year,year_level,Reading,Writing,Spelling,Grammar,Numeracy,school_participation_pct,australian_participation_pct
0,44370,Abbotsford Primary School,2024,Year 3,455.0,451.0,439.0,446.0,443.0,92,95
1,44370,Abbotsford Primary School,2024,Year 5,545.0,513.0,502.0,537.0,544.0,92,95
2,44727,Aberfeldie Primary School,2023,Year 3,450.0,443.0,435.0,449.0,454.0,100,95
3,44727,Aberfeldie Primary School,2023,Year 5,538.0,502.0,501.0,521.0,511.0,100,95
4,44727,Aberfeldie Primary School,2024,Year 3,408.0,424.0,407.0,411.0,424.0,96,95
5,44727,Aberfeldie Primary School,2024,Year 5,523.0,507.0,483.0,506.0,507.0,96,95
6,45704,Academy of Mary Immaculate,2023,Year 7,566.0,572.0,559.0,561.0,550.0,100,95
7,45704,Academy of Mary Immaculate,2023,Year 9,604.0,625.0,595.0,598.0,579.0,100,95
8,45704,Academy of Mary Immaculate,2024,Year 7,558.0,583.0,545.0,561.0,535.0,98,95
9,45704,Academy of Mary Immaculate,2024,Year 9,597.0,622.0,580.0,597.0,577.0,98,95



Summary statistics:
          Reading     Writing    Spelling     Grammar    Numeracy
count  224.000000  224.000000  224.000000  224.000000  224.000000
mean   489.754464  490.236607  478.651786  486.723214  483.741071
std     71.089077   64.789638   68.270592   69.011237   68.218358
min    304.000000  362.000000  304.000000  308.000000  318.000000
25%    433.750000  441.000000  422.000000  433.750000  428.750000
50%    496.000000  483.000000  480.000000  490.500000  483.000000
75%    545.000000  535.750000  535.250000  537.000000  538.000000
max    645.000000  628.000000  605.000000  644.000000  632.000000


In [ ]:
#Transform to Wide Format and Export to CSV

if 'naplan_df' in globals() and not naplan_df.empty:
    print("="*60)
    print("="*60)
    print("\nOriginal format (long):")
    print(f"  Rows: {len(naplan_df)}")
    print(f"  One row per: School × Year × Year Level")
    print()
    
    
    
    
    naplan_df['row_id'] = naplan_df['school_id'].astype(str) + '_' + naplan_df['year'].astype(str)
    
    base_cols = naplan_df[['school_id', 'school_name', 'year']].drop_duplicates()
    
    # Initialize the wide dataframe
    wide_df = base_cols.copy()
    
    # For each year level, create columns
    subjects = ['Reading', 'Writing', 'Spelling', 'Grammar', 'Numeracy']
    year_levels = ['Year 3', 'Year 5', 'Year 7', 'Year 9']
    
    for year_level in year_levels:
        # Filter data for this year level
        level_data = naplan_df[naplan_df['year_level'] == year_level].copy()
        
        if not level_data.empty:
            # Rename columns to include year level
            rename_dict = {
                'school_id': 'school_id',
                'year': 'year'
            }
            
            for subject in subjects:
                if subject in level_data.columns:
                    level_data[f'{year_level} {subject}'] = level_data[subject]
            
            
            cols_to_keep = ['school_id', 'year'] + [f'{year_level} {subject}' for subject in subjects if subject in level_data.columns]
            level_data = level_data[cols_to_keep].drop_duplicates()
            
            # Merge with wide_df
            wide_df = wide_df.merge(level_data, on=['school_id', 'year'], how='left')
    
    # Add participation percentages (they should be the same across year levels for a given school-year)
    participation_data = naplan_df[['school_id', 'year', 'school_participation_pct', 'australian_participation_pct']].drop_duplicates()
    wide_df = wide_df.merge(participation_data, on=['school_id', 'year'], how='left')
    
    # Rename columns to match the desired format
    wide_df = wide_df.rename(columns={
        'school_id': 'ACARA School ID',
        'year': 'Year Page Visited',
        'school_participation_pct': 'Participation School Percentage',
        'australian_participation_pct': 'Participation Australian Percentage'
    })
    
    # Remove school_name from final output
    if 'school_name' in wide_df.columns:
        wide_df = wide_df.drop('school_name', axis=1)
    
    # Reorder columns to match the image: School ID, Year, Year 3 subjects, Year 5 subjects, Year 7 subjects, Year 9 subjects, Participation
    column_order = ['ACARA School ID', 'Year Page Visited']
    
    # Add year level columns in order
    for year_level in year_levels:
        for subject in subjects:
            col_name = f'{year_level} {subject}'
            if col_name in wide_df.columns:
                column_order.append(col_name)
    
    # Add participation columns
    column_order.extend(['Participation School Percentage', 'Participation Australian Percentage'])
    
    # Reorder
    wide_df = wide_df[[col for col in column_order if col in wide_df.columns]]
    
    # Sort by school ID and year
    wide_df = wide_df.sort_values(['ACARA School ID', 'Year Page Visited'])
    
    print("\nTransformed format (wide):")
    print(f"  Rows: {len(wide_df)}")
    print(f"  One row per: School × Year")
    print(f"  Columns: {len(wide_df.columns)}")
    print()
    
    # Display sample
    print("Sample of transformed data:")
    display(wide_df.head(10))
    
    # Create filename with timestamp
    output_filename = OUTPUT_CSV
    
    # Export to CSV
    wide_df.to_csv(output_filename, index=False)
    
    print("CSV EXPORT SUCCESSFUL")
    print(f"   Records: {len(wide_df):,}")
    print(f"   Schools: {wide_df['ACARA School ID'].nunique()}")
    print(f"   Years: {sorted(wide_df['Year Page Visited'].unique())}")
    print(f"   Columns: {wide_df.columns.tolist()}")
    
    # Also save summary
    summary_file = output_filename.replace('.csv', '_summary.txt')
    with open(summary_file, 'w') as f:
        f.write("NAPLAN Data Collection Summary\n")
        f.write("="*60 + "\n\n")
        f.write(f"Collection Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Output Format: Wide (one row per School × Year)\n")
        f.write(f"Total Records: {len(wide_df):,}\n")
        f.write(f"Unique Schools: {wide_df['ACARA School ID'].nunique()}\n")
        f.write(f"Years: {sorted(wide_df['Year Page Visited'].unique())}\n")
        f.write(f"\nColumns ({len(wide_df.columns)}):\n")
        f.write(f"  {', '.join(wide_df.columns)}\n")
        f.write(f"\nData Quality:\n")
        f.write(f"  Missing values:\n")
        for col in wide_df.columns:
            missing = wide_df[col].isna().sum()
            if missing > 0:
                pct = (missing / len(wide_df)) * 100
                f.write(f"    {col}: {missing} ({pct:.1f}%)\n")
    
    print(f"\n Summary saved to: {summary_file}")
else:
    print("No data to export")



Original format (long):
  Rows: 224
  One row per: School × Year × Year Level


Transformed format (wide):
  Rows: 93
  One row per: School × Year
  Columns: 24

Sample of transformed data:


,ACARA School ID,Year Page Visited,Year 3 Reading,Year 3 Writing,Year 3 Spelling,Year 3 Grammar,Year 3 Numeracy,Year 5 Reading,Year 5 Writing,Year 5 Spelling,...,Year 7 Spelling,Year 7 Grammar,Year 7 Numeracy,Year 9 Reading,Year 9 Writing,Year 9 Spelling,Year 9 Grammar,Year 9 Numeracy,Participation School Percentage,Participation Australian Percentage
35,44030,2023,344.0,393.0,355.0,335.0,318.0,445.0,481.0,451.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88,95
36,44030,2024,421.0,395.0,386.0,405.0,434.0,454.0,457.0,445.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,95
58,44031,2023,420.0,455.0,416.0,415.0,414.0,472.0,476.0,472.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,95
59,44031,2024,408.0,424.0,389.0,388.0,386.0,456.0,467.0,437.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,95
43,44166,2023,369.0,394.0,368.0,370.0,393.0,505.0,445.0,458.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,95
44,44166,2024,304.0,373.0,304.0,308.0,341.0,471.0,457.0,453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,95
47,44196,2023,391.0,413.0,375.0,384.0,391.0,474.0,493.0,460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,95
48,44196,2024,407.0,422.0,396.0,397.0,401.0,470.0,471.0,476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,95
33,44224,2023,458.0,458.0,439.0,458.0,456.0,536.0,521.0,517.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,95
34,44224,2024,420.0,433.0,425.0,434.0,430.0,548.0,524.0,518.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,95



📄 CSV EXPORT SUCCESSFUL
   Records: 93
   Schools: 49
   Years: [np.int64(2023), np.int64(2024)]
   Columns: ['ACARA School ID', 'Year Page Visited', 'Year 3 Reading', 'Year 3 Writing', 'Year 3 Spelling', 'Year 3 Grammar', 'Year 3 Numeracy', 'Year 5 Reading', 'Year 5 Writing', 'Year 5 Spelling', 'Year 5 Grammar', 'Year 5 Numeracy', 'Year 7 Reading', 'Year 7 Writing', 'Year 7 Spelling', 'Year 7 Grammar', 'Year 7 Numeracy', 'Year 9 Reading', 'Year 9 Writing', 'Year 9 Spelling', 'Year 9 Grammar', 'Year 9 Numeracy', 'Participation School Percentage', 'Participation Australian Percentage']

📄 Summary saved to: Final_Result_summary.txt

